## Imports

In [1]:
!pip install pytickersymbols

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import bs4 as bs
from pytickersymbols import PyTickerSymbols

## 

## Dataset Init

In [3]:
def get_tickers():
    stock_data = PyTickerSymbols()
    dow_yahoo = stock_data.get_dow_jones_nyc_yahoo_tickers()
    dow_yahoo.remove('CAT')
    sp100_yahoo = stock_data.get_sp_100_nyc_yahoo_tickers()
    sp100_yahoo.remove('MNSLV')
    sp100_yahoo.remove('BOAPL')
    sp100_yahoo.remove('NEEXU')
    sp100_yahoo.remove('BMYMP')
    sp100_yahoo.remove('XON')
    return set(dow_yahoo + sp100_yahoo)

In [4]:
tickers = get_tickers()
df = pd.DataFrame(tickers, columns=['Ticker'])
df

,Ticker
0,TRV
1,ORCL
2,CVX
3,AVGO
4,WMT
...,...
99,PYPL
100,GD
101,NEE
102,ADBE


In [5]:
def get_city(ticker):
    try:
        return yf.Ticker(ticker).info['city']
    except:
        return 'None'

In [6]:
# add city
df['City'] = df['Ticker'].apply(lambda x: get_city(x))
df

,Ticker,City
0,TRV,New York
1,ORCL,Austin
2,CVX,San Ramon
3,AVGO,San Jose
4,WMT,Bentonville
...,...,...
99,PYPL,San Jose
100,GD,Reston
101,NEE,Juno Beach
102,ADBE,San Jose


In [7]:
def get_country(ticker):
    try:
        return yf.Ticker(ticker).info['country']
    except:
        return 'None'

In [8]:
# add country
df['Country'] = df['Ticker'].apply(lambda x: get_country(x))
df

,Ticker,City,Country
0,TRV,New York,United States
1,ORCL,Austin,United States
2,CVX,San Ramon,United States
3,AVGO,San Jose,United States
4,WMT,Bentonville,United States
...,...,...,...
99,PYPL,San Jose,United States
100,GD,Reston,United States
101,NEE,Juno Beach,United States
102,ADBE,San Jose,United States


In [9]:
def get_industry(ticker):
    try:
        return yf.Ticker(ticker).info['industry']
    except:
        return 'None'

In [10]:
# add industry
df['Industry'] = df['Ticker'].apply(lambda x: get_industry(x))
df

,Ticker,City,Country,Industry
0,TRV,New York,United States,Insurance - Property & Casualty
1,ORCL,Austin,United States,Software - Infrastructure
2,CVX,San Ramon,United States,Oil & Gas Integrated
3,AVGO,San Jose,United States,Semiconductors
4,WMT,Bentonville,United States,Discount Stores
...,...,...,...,...
99,PYPL,San Jose,United States,Credit Services
100,GD,Reston,United States,Aerospace & Defense
101,NEE,Juno Beach,United States,Utilities - Regulated Electric
102,ADBE,San Jose,United States,Software - Infrastructure


In [11]:
def get_sector(ticker):
    try:
        return yf.Ticker(ticker).info['sector']
    except:
        return 'None'

In [12]:
# add sector
df['Sector'] = df['Ticker'].apply(lambda x: get_sector(x))
df

,Ticker,City,Country,Industry,Sector
0,TRV,New York,United States,Insurance - Property & Casualty,Financial Services
1,ORCL,Austin,United States,Software - Infrastructure,Technology
2,CVX,San Ramon,United States,Oil & Gas Integrated,Energy
3,AVGO,San Jose,United States,Semiconductors,Technology
4,WMT,Bentonville,United States,Discount Stores,Consumer Defensive
...,...,...,...,...,...
99,PYPL,San Jose,United States,Credit Services,Financial Services
100,GD,Reston,United States,Aerospace & Defense,Industrials
101,NEE,Juno Beach,United States,Utilities - Regulated Electric,Utilities
102,ADBE,San Jose,United States,Software - Infrastructure,Technology


In [13]:
# Here I am not using the idea we wrote in preliminaries
# For the beginning I just want to test my idea which is simplier and I just want get some results if it is possible
def rsi(close, periods = 14):
    
    close_delta = close.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

def rsi_for_ticker(ticker):
    history_data = yf.Ticker(ticker).history(start='2019-01-01', end='2020-04-01', interval='1mo')
    rsi_vec = rsi(history_data['Close'])
    return rsi_vec[-1]

df['RSI'] = df['Ticker'].apply(lambda x: rsi_for_ticker(x))
df

,Ticker,City,Country,Industry,Sector,RSI
0,TRV,New York,United States,Insurance - Property & Casualty,Financial Services,25.214000
1,ORCL,Austin,United States,Software - Infrastructure,Technology,42.124874
2,CVX,San Ramon,United States,Oil & Gas Integrated,Energy,22.004684
3,AVGO,San Jose,United States,Semiconductors,Technology,39.318903
4,WMT,Bentonville,United States,Discount Stores,Consumer Defensive,63.841532
...,...,...,...,...,...,...
99,PYPL,San Jose,United States,Credit Services,Financial Services,44.492201
100,GD,Reston,United States,Aerospace & Defense,Industrials,28.189380
101,NEE,Juno Beach,United States,Utilities - Regulated Electric,Utilities,69.144791
102,ADBE,San Jose,United States,Software - Infrastructure,Technology,62.335515


In [14]:
def get_relative_change_vector(ticker):
    prices = yf.Ticker(ticker).history(start='2020-04-01', end = '2020-04-30', interval='1d')['Close']
    vec = list()
    for i in range(len(prices)-1):
        vec.append((prices[i+1]-prices[i])/ prices[i])
    return vec

In [15]:
df['Ticker'].apply(lambda x: print(f'{x}: {get_relative_change_vector(x)}'))

TRV: [0.020903292382634598, -0.033954237381069875, 0.05421257172470229, 0.01818542218421865, 0.04127819316127887, 0.04574016462620663, -0.03025314434085003, 0.01503458016421068, -0.054156591456732676, -0.023098659992953727, 0.052699972607627814, -0.031312301110951325, 0.0, 9.829411338587773e-05, -0.013164588656397067, 0.003683636304922971, 0.020531758215493877, 0.013606603140125156, 0.01639661903491247]
ORCL: [0.022377365432651612, -0.008032053994309835, 0.04230761247414947, -0.014177728951695317, 0.03186863830886027, 0.0201421176603861, -0.003949005994012056, 0.01944514524746529, -0.015555709932913078, 0.010158235186280092, 0.017132163195880516, -0.012999070809184736, -0.04822842039639064, 0.018709720175723196, -0.005739443480877975, 0.020011639376174145, 0.006791141375466559, -0.003185264583899872, 0.012781923304214142]
CVX: [0.11026805282514532, -0.013268189216119941, 0.07029677986123314, 0.006592681255660731, 0.06253096875853419, -0.01942308599707371, 0.007116797256812023, -0.00353

PG: [0.046373340249025806, 0.005944136246133421, 0.023722513622321036, -0.04278058023373215, 0.02066146218772702, -0.003822716343330323, 0.011250434359744124, 0.04320831925193893, 0.002149595120495423, 0.0023098405177117516, 0.02625513346583196, -0.03280132255509028, -0.007628581843448162, -0.0023396323685107284, 0.006669054336442646, -0.005192694152044881, -0.011197101960931384, -0.004767904693133072, 0.001625479831182006]
GE: [-0.019886213670407108, -0.024637651888186778, 0.07429422282947365, -0.02766252345053023, 0.03840671333127108, -0.02191788353496252, -0.01680652149246491, -0.01282053318143542, -0.062048933465527166, -0.04000027062106295, 0.09615388242174554, -0.04824548575232813, -0.004608333952554026, -0.007716146883504219, 0.013997010025661378, -0.039877385702674385, 0.027156518542000845, 0.057543038677013415, -0.03235327202997431]
QCOM: [0.032169596796832844, -0.04101711001280779, 0.08768985893175257, 0.025087945149729753, 0.008249794327320323, -0.02386464404073885, 0.024028

MO: [-0.03695816764446389, 0.032854335893691564, -0.00347491299946287, 0.03218904442410413, 0.042879279898376296, 0.019436929828304573, -0.011488779551359077, 0.041543064531242936, -0.02872733724721769, -0.012466400243258827, 0.011138674716987897, -0.0435740802947031, -0.03352957001704279, 0.0013242189422377495, 0.016926722054465052, 0.025227197087608046, 0.008371807483320781, 0.024150696897174202, -0.0058953449795679964]
GM: [-0.0555555221361874, -0.008246275980663114, 0.083702883076709, 0.08951398793414747, 0.08591543307376427, 0.04020761321936401, -0.043640829107294256, -0.0013040137928893108, -0.057441104952580285, -0.03647279871525344, 0.07714428703169207, -0.004448448785158941, -0.050938380363869856, 0.0028249090925289775, 0.010328701148130054, 0.019981422479589315, 0.02277894789439709, -0.01202655296099163, 0.07213690914077021]
SPG: [-0.06441330756391844, 0.061577109356687094, 0.15710587419736224, 0.1261562729729746, 0.03334426308515714, 0.08361138180853696, -0.04210043726909437

BRK-B: [0.02148336939391256, -0.007733819744566184, 0.03869019455512595, 5.395436482162234e-05, 0.031093087756199393, 0.01481598823329324, -0.02615555180155891, 0.02209036432995229, -0.02264950275676823, -0.003234876102554077, 0.01723765757950445, -0.012813791774826238, -0.02792055243687914, 0.016895608108248587, -0.0050916332882362985, 0.0047944802361573535, 0.005039687003401159, -0.001813782057517314, 0.013306998493691968]
AMZN: [0.005834205106051261, -0.006378882191073191, 0.04772922477644708, 0.007013456003121437, 0.01560946178407676, -0.00011748445832980121, 0.06173506614263288, 0.052769449861283735, 0.010668695718768674, 0.043554542393708595, -0.013782136130354523, 0.007835757606907894, -0.027360330959314613, 0.015192527091639185, 0.015214823484082759, 0.00448851118687423, -0.014197861710760542, -0.026060611042559304, 0.025336164749658024]
LOW: [0.03034052394952025, -0.007965011234789992, 0.0799266483560369, 0.02467054637825864, 0.037269093755115494, 0.010175109370334084, -0.0011

CAT: [0.04840601308546325, -0.01773185345323401, 0.05023121854276617, 0.01237240134028218, 0.044947449706867856, -0.01860293165343385, -0.0870990097279109, 0.020501231147364643, -0.04249661925353061, 0.015152904169305266, 0.03663427347184319, -0.014617463066792085, -0.04144844233949126, 0.00719151151163112, 0.020517078089048985, 0.010007938460438553, 0.010171844464009618, 0.002257002786608697, 0.03984059923501218]
NKE: [0.011485530742742948, -0.015972038709751158, 0.07316749825849841, -0.005553378491854672, 0.013545618992933195, 0.01746756097784498, -0.026846386189720663, 0.03563812098627643, -0.02778081196548019, 0.01481672001838487, 0.04183077831995965, -0.02235572755684418, -0.030716699941833958, 0.04190124753602622, -0.015996295355059458, 0.011677077880301432, 0.011316142703990173, -0.006378019612849871, -0.008220504836294605]
AMD: [0.019010577965081803, -0.04270625880417776, 0.11575487878640499, 0.0008417700997898144, 0.02586205859378039, -0.008403358059409824, 0.05291437581800536

ADBE: [0.009129843225702323, -0.03405055400208892, 0.08691809136086724, -0.031961934167790924, 0.026705079453577973, 0.004792293228923022, 0.006118549152067161, 0.06274753001767669, -0.024121846071290434, 0.03052180059131882, 0.00411430724922023, 0.0022377134173055275, -0.04607403857418989, 0.023648132193184186, -0.00415712189968192, 0.026030984027887227, 0.012786962564484014, -0.04318504388226327, 0.047143501710064255]
CMCSA: [0.060148235219297415, -0.012220187519240677, 0.06715773740061542, 0.019597039209078636, 0.020032451547288336, 0.008492602662988477, -0.02131570949088327, 0.03576227199941102, -0.03530647125913146, 0.01803037985875784, 0.006608372271747638, -0.022846564844580346, -0.04031178771998596, 0.0005600693877294966, 0.010075489832843029, 0.02964804712016608, 0.02825635883341807, -0.003402318118969848, 0.02415997650326588]


0      None
1      None
2      None
3      None
4      None
       ... 
99     None
100    None
101    None
102    None
103    None
Name: Ticker, Length: 104, dtype: object

In [16]:
def SMA(data, ndays): 
    SMA = data['Close'].rolling(ndays).mean()
    return SMA

In [17]:
def get_macd(ticker):
    df1 = yf.Ticker(ticker).history(start='2018-06-01', end='2020-04-01', interval='1mo')
    macd = SMA(df1, 7)[-1] - SMA(df1, 15)[-1]
    return macd

df['MACD'] = df['Ticker'].apply(lambda x: get_macd(x))
df

,Ticker,City,Country,Industry,Sector,RSI,MACD
0,TRV,New York,United States,Insurance - Property & Casualty,Financial Services,25.214000,-4.868960
1,ORCL,Austin,United States,Software - Infrastructure,Technology,42.124874,-0.079800
2,CVX,San Ramon,United States,Oil & Gas Integrated,Energy,22.004684,-4.549823
3,AVGO,San Jose,United States,Semiconductors,Technology,39.318903,4.656969
4,WMT,Bentonville,United States,Discount Stores,Consumer Defensive,63.841532,6.474634
...,...,...,...,...,...,...,...
99,PYPL,San Jose,United States,Credit Services,Financial Services,44.492201,0.021142
100,GD,Reston,United States,Aerospace & Defense,Industrials,28.189380,-2.138165
101,NEE,Juno Beach,United States,Utilities - Regulated Electric,Utilities,69.144791,6.025250
102,ADBE,San Jose,United States,Software - Infrastructure,Technology,62.335515,20.564762


In [18]:
def get_price_change(ticker):
    history_data = yf.Ticker(ticker).history(start='2019-01-01', end='2020-05-01', interval='1mo')
    close = history_data['Close']
    return close[-1] - close[-2]

df['Price Change'] = df['Ticker'].apply(lambda x: get_price_change(x))
df

,Ticker,City,Country,Industry,Sector,RSI,MACD,Price Change
0,TRV,New York,United States,Insurance - Property & Casualty,Financial Services,25.214000,-4.868960,2.316551
1,ORCL,Austin,United States,Software - Infrastructure,Technology,42.124874,-0.079800,4.367290
2,CVX,San Ramon,United States,Oil & Gas Integrated,Energy,22.004684,-4.549823,16.682869
3,AVGO,San Jose,United States,Semiconductors,Technology,39.318903,4.656969,34.590164
4,WMT,Bentonville,United States,Discount Stores,Consumer Defensive,63.841532,6.474634,7.979797
...,...,...,...,...,...,...,...,...
99,PYPL,San Jose,United States,Credit Services,Financial Services,44.492201,0.021142,27.260002
100,GD,Reston,United States,Aerospace & Defense,Industrials,28.189380,-2.138165,-1.537231
101,NEE,Juno Beach,United States,Utilities - Regulated Electric,Utilities,69.144791,6.025250,-2.202198
102,ADBE,San Jose,United States,Software - Infrastructure,Technology,62.335515,20.564762,35.400024


In [19]:
def get_relative_change_vector(ticker):
    prices = yf.Ticker(ticker).history(start='2020-04-01', end = '2020-04-30', interval='1d')['Close']
    vec = list()
    for i in range(len(prices)-1):
        vec.append((prices.iloc[i+1]-prices.iloc[i])/ prices.iloc[i])
    vec = tuple(vec)
    return vec

In [20]:
price_vectors = {x: get_relative_change_vector(x) for x in df["Ticker"]}
# df['Ticker'].apply(lambda x: print(f'{x}: {get_relative_change_vector(x)}'))
display(price_vectors)

{'TRV': (0.020903468042389135,
  -0.033954319631799824,
  0.05421257172470229,
  0.01818533858230721,
  0.04127811444184287,
  0.045740329547378396,
  -0.03025329515037293,
  0.015034815773819937,
  -0.05415651070262035,
  -0.02309881823465419,
  0.052700055514323295,
  -0.0313123774011497,
  0.0,
  9.82128113896942e-05,
  -0.013164427138575154,
  0.003683553622993085,
  0.020531429910911267,
  0.013607090066808165,
  0.01639637839717677),
 'ORCL': (0.022377367294550466,
  -0.008032217414813539,
  0.04230778697199565,
  -0.014177651355401416,
  0.03186863830886027,
  0.020141962903751354,
  -0.00394870319092597,
  0.019444913832209733,
  -0.015555711094904808,
  0.010158235957077632,
  0.017132164482780664,
  -0.012998997918367235,
  -0.04822849521982977,
  0.018709800261504664,
  -0.005739443480877975,
  0.02001156175963524,
  0.006791294079797116,
  -0.0031853399236500967,
  0.012781847482201162),
 'CVX': (0.11026838585746535,
  -0.013268655675644074,
  0.07029703453253827,
  0.00659

In [21]:
df["Ticker_diff"] = df["Ticker"].apply(lambda x: get_relative_change_vector(x))
display(df)

,Ticker,City,Country,Industry,Sector,RSI,MACD,Price Change,Ticker_diff
0,TRV,New York,United States,Insurance - Property & Casualty,Financial Services,25.214000,-4.868960,2.316551,"(0.020903381120983645, -0.033953981956098916, ..."
1,ORCL,Austin,United States,Software - Infrastructure,Technology,42.124874,-0.079800,4.367290,"(0.02237728222809813, -0.00803205464798581, 0...."
2,CVX,San Ramon,United States,Oil & Gas Integrated,Energy,22.004684,-4.549823,16.682869,"(0.11026845821282452, -0.013268420888280424, 0..."
3,AVGO,San Jose,United States,Semiconductors,Technology,39.318903,4.656969,34.590164,"(0.06005449803914246, -0.011811290610755108, 0..."
4,WMT,Bentonville,United States,Discount Stores,Consumer Defensive,63.841532,6.474634,7.979797,"(0.039513038816252115, 0.006995193220550383, 0..."
...,...,...,...,...,...,...,...,...,...
99,PYPL,San Jose,United States,Credit Services,Financial Services,44.492201,0.021142,27.260002,"(0.02341868520430923, -0.012082947968131124, 0..."
100,GD,Reston,United States,Aerospace & Defense,Industrials,28.189380,-2.138165,-1.537231,"(0.004081731812503028, -0.017511040173188545, ..."
101,NEE,Juno Beach,United States,Utilities - Regulated Electric,Utilities,69.144791,6.025250,-2.202198,"(0.037666632532425394, -0.02124101538005172, 0..."
102,ADBE,San Jose,United States,Software - Infrastructure,Technology,62.335515,20.564762,35.400024,"(0.009129843225702323, -0.03405055400208892, 0..."


In [22]:
df.to_pickle('df_stocks.pkl')

## Experiments

In [1]:
import pandas as pd
df = pd.read_pickle('df_stocks.pkl')
df

,Ticker,City,Country,Industry,Sector,RSI,MACD,Price Change,Ticker_diff
0,TRV,New York,United States,Insurance - Property & Casualty,Financial Services,25.214000,-4.868960,2.316551,"(0.020903381120983645, -0.033953981956098916, ..."
1,ORCL,Austin,United States,Software - Infrastructure,Technology,42.124874,-0.079800,4.367290,"(0.02237728222809813, -0.00803205464798581, 0...."
2,CVX,San Ramon,United States,Oil & Gas Integrated,Energy,22.004684,-4.549823,16.682869,"(0.11026845821282452, -0.013268420888280424, 0..."
3,AVGO,San Jose,United States,Semiconductors,Technology,39.318903,4.656969,34.590164,"(0.06005449803914246, -0.011811290610755108, 0..."
4,WMT,Bentonville,United States,Discount Stores,Consumer Defensive,63.841532,6.474634,7.979797,"(0.039513038816252115, 0.006995193220550383, 0..."
...,...,...,...,...,...,...,...,...,...
99,PYPL,San Jose,United States,Credit Services,Financial Services,44.492201,0.021142,27.260002,"(0.02341868520430923, -0.012082947968131124, 0..."
100,GD,Reston,United States,Aerospace & Defense,Industrials,28.189380,-2.138165,-1.537231,"(0.004081731812503028, -0.017511040173188545, ..."
101,NEE,Juno Beach,United States,Utilities - Regulated Electric,Utilities,69.144791,6.025250,-2.202198,"(0.037666632532425394, -0.02124101538005172, 0..."
102,ADBE,San Jose,United States,Software - Infrastructure,Technology,62.335515,20.564762,35.400024,"(0.009129843225702323, -0.03405055400208892, 0..."


In [2]:
df = df.drop(['Ticker', 'MACD', 'Price Change'], axis=1)
df

,City,Country,Industry,Sector,RSI,Ticker_diff
0,New York,United States,Insurance - Property & Casualty,Financial Services,25.214000,"(0.020903381120983645, -0.033953981956098916, ..."
1,Austin,United States,Software - Infrastructure,Technology,42.124874,"(0.02237728222809813, -0.00803205464798581, 0...."
2,San Ramon,United States,Oil & Gas Integrated,Energy,22.004684,"(0.11026845821282452, -0.013268420888280424, 0..."
3,San Jose,United States,Semiconductors,Technology,39.318903,"(0.06005449803914246, -0.011811290610755108, 0..."
4,Bentonville,United States,Discount Stores,Consumer Defensive,63.841532,"(0.039513038816252115, 0.006995193220550383, 0..."
...,...,...,...,...,...,...
99,San Jose,United States,Credit Services,Financial Services,44.492201,"(0.02341868520430923, -0.012082947968131124, 0..."
100,Reston,United States,Aerospace & Defense,Industrials,28.189380,"(0.004081731812503028, -0.017511040173188545, ..."
101,Juno Beach,United States,Utilities - Regulated Electric,Utilities,69.144791,"(0.037666632532425394, -0.02124101538005172, 0..."
102,San Jose,United States,Software - Infrastructure,Technology,62.335515,"(0.009129843225702323, -0.03405055400208892, 0..."


In [3]:
import sys
sys.path.append('emm')
from emm import EMM


In [4]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [5]:
target_columns = ['Ticker_diff']
target_col2 = ['City', 'City']
clf = EMM.EMM(width=8, depth=4, evaluation_metric='correlation', strategy='maximize')
clf.search(df, target_cols=target_columns)
#clf.visualise(subgroups=5, cols=3, target_columns=target_col2) 

In correlation func
In correlation func
City = Thousand Oaks 0.0331482586853804 (1)
In correlation func
City = Stamford 0.07601837977037444 (2)
In correlation func
City = Dallas 0.09017171288812299 (2)
In correlation func
City = McLean 0.04138357559812702 (1)
In correlation func
City = Midland 0.04441174733789916 (1)
In correlation func
City = Beaverton 0.048249091396387736 (1)
In correlation func
City = Reston 0.04805588967665126 (1)
In correlation func
Industry = Insurance - Property & <br> Casualty 0.04489051010921957 (1)
In correlation func
Industry = Beverages - <br> Non-Alcoholic 0.08026042205618765 (2)
In correlation func
Industry = REIT - Specialty 0.03690658924195474 (1)
In correlation func
Industry = Banks - Regional 0.04520505060083746 (1)
In correlation func
Industry = Insurance - Life 0.04581332445433648 (1)
In correlation func
Industry = Pharmaceutical <br> Retailers 0.030691918429941012 (1)
In correlation func
Sector = Consumer Cyclical 0.2928316916010211 (10)
In correla

In correlation func
Sector = Financial Services AND <br> Industry = Asset Management 0.07685249482903103 (2)
In correlation func
Sector = Financial Services AND 31 < <br> RSI <= 34 0.08112208869881017 (2)
In correlation func
Sector = Technology AND City = <br> Austin 0.048699864452760186 (1)
In correlation func
Sector = Technology AND City = San <br> Diego 0.03856414022838104 (1)
In correlation func
Sector = Technology AND Country = <br> United States 0.32562877922765143 (13)
In correlation func
Sector = Technology AND Industry = <br> Software - Application 0.06765367902153883 (2)
In correlation func
Sector = Technology AND 42 < RSI <= 42 0.04794265718188759 (1)
In correlation func
Sector = Healthcare AND City = New <br> Brunswick 0.031123438803847953 (1)
In correlation func
Sector = Healthcare AND City = <br> Washington 0.03730231712972318 (1)
In correlation func
Sector = Healthcare AND City = <br> Rahway 0.04277615169695634 (1)
In correlation func
Sector = Healthcare AND Industry = <

In correlation func
Sector = Financial Services AND <br> City = Purchase 0.04893988402915512 (1)
In correlation func
Sector = Financial Services AND <br> City = Westlake 0.04378562370414748 (1)
In correlation func
Sector = Financial Services AND <br> City = Charlotte 0.04497791526030654 (1)
In correlation func
Sector = Financial Services AND <br> City = Omaha 0.05134608450148991 (1)
In correlation func
Sector = Financial Services AND <br> Industry = Insurance - Property & Casualty 0.04489051010921957 (1)
In correlation func
Sector = Financial Services AND <br> Industry = Capital Markets 0.1480087462019286 (4)
In correlation func
Sector = Financial Services AND <br> Industry = Banks - Regional 0.04520505060083746 (1)
In correlation func
Sector = Financial Services AND <br> Industry = Insurance - Diversified 0.07520073297923999 (2)
In correlation func
Sector = Financial Services AND <br> Industry = Insurance - Life 0.04581332445433648 (1)
In correlation func
Sector = Financial Services A

In correlation func
Sector = Technology AND 53 < RSI <= 60 0.035949161342272114 (1)
In correlation func
Sector = Healthcare AND City = <br> Waltham 0.04208380219670985 (1)
In correlation func
Sector = Healthcare AND City = <br> Rahway 0.04277615169695634 (1)
In correlation func
Sector = Healthcare AND Industry = <br> Drug Manufacturers - General 0.21064059611290145 (8)
In correlation func
Sector = Healthcare AND 45 < RSI <= 46 0.06889822222468833 (2)
In correlation func
Sector = Industrials AND City = <br> Atlanta 0.03417564943303336 (1)
In correlation func
Sector = Industrials AND City = <br> Omaha 0.04511647068104457 (1)
In correlation func
Sector = Industrials AND Industry = <br> Specialty Industrial Machinery 0.06906315876605414 (2)
In correlation func
Sector = Industrials AND 31 < RSI <= <br> 33 0.042366651250390845 (1)
In correlation func
42 < RSI <= 45 AND City = Dublin 0.05143113321229548 (1)
In correlation func
42 < RSI <= 45 AND City = Atlanta 0.04871281920788393 (1)
In corre

Sector = Financial Services 0.4509438062361403 (20)
Sector = Technology 0.3613660596732916 (15)
Sector = Healthcare 0.3531921651155418 (15)
Sector = Industrials 0.3088479269193064 (11)
42 < RSI <= 45 0.3076349360131479 (10)
38 < RSI <= 42 0.33219746574582093 (11)
53 < RSI <= 63 0.22451611147403192 (10)
30 < RSI <= 34 0.30762375811777387 (10)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=88720f49-c7ba-4446-9879-24b162efc293' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>